In [1]:
import pandas as pd

In [2]:
# Read the data
artistas = pd.read_csv('Artistas.csv', sep=',')
artistas = artistas.drop_duplicates()

entradas = pd.read_csv('Entradas Cortesia.csv', sep=',')
entradas = entradas.drop_duplicates()

eventos = pd.read_csv('Eventos.csv', sep=',')
eventos = eventos.drop_duplicates()

hosp_traslados = pd.read_csv('Hospedajes y Traslados.csv', sep=',')
hosp_traslados = hosp_traslados.drop_duplicates()

productoras = pd.read_csv('Productoras.csv', sep=',')
productoras = productoras.drop_duplicates()

tour = pd.read_csv('Tour.csv', sep=',')
tour = tour.drop_duplicates()

# replace the 'nombre artistico' column in artistas table with 'Artista'
artistas = artistas.rename(columns={'Nombre artistico': 'Artista'})
entradas = entradas.rename(columns={'evento': 'Evento'})
entradas = entradas.rename(columns={'artista': 'Artista'})

In [3]:
# Add a id column to artistas
artistas['id'] = range(1, len(artistas) + 1)
# Add a id column to entradas table
entradas['id'] = range(1, len(entradas) + 1)
# Add a id column to eventos table
eventos['id'] = range(1, len(eventos) + 1)

In [4]:
# agregar columna id_artista a entradas
entradas = entradas.merge(artistas[['id', 'Artista']], on='Artista', how='left')
# renombrar id_x a id_entrada y id_y a id_artista
entradas = entradas.rename(columns={'id_x': 'id_entrada', 'id_y': 'id_artista'})
# agregar columna id_artista a eventos
eventos = eventos.merge(artistas[['id', 'Artista']], on='Artista', how='left')
# renombrar id_x a id_evento y id_y a id_artista
eventos = eventos.rename(columns={'id_x': 'id_evento', 'id_y': 'id_artista'})
#quitar columna artista de eventos
eventos = eventos.drop(columns=['Artista'])
#quitar columna artista de entradas
entradas = entradas.drop(columns=['Artista'])
# agregar columna id_evento a entradas
entradas = entradas.merge(eventos[['id_evento', 'Evento']], on='Evento', how='left')
# quitamos la columna evento de entradas
entradas = entradas.drop(columns=['Evento'])